<a href="https://colab.research.google.com/github/robbybrodie/time_as_computation_cost/blob/main/notebooks/bandgaps_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bandgaps Experiment: DoF Law Fitting

This experiment fits microphysical degrees of freedom (DoF) laws and beta parameters from synthetic data, comparing our computational-capacity model against baseline models.

## Setup: Clone Repository and Install Dependencies

In [ ]:
# Clone and setup (idempotent)
import os, sys, subprocess, shutil, pathlib
REPO_URL = "https://github.com/robbybrodie/time_as_computation_cost.git"
REPO_NAME = "time_as_computation_cost"

if not pathlib.Path(REPO_NAME).exists():
    !git clone $REPO_URL

%cd $REPO_NAME

# Install package
if (pathlib.Path("pyproject.toml")).exists():
    !pip install -e .

# Set random seed for reproducibility
import numpy as np, random
np.random.seed(42)
random.seed(42)

## Run Bandgaps Experiment

In [ ]:
from experiments.run_bandgaps import main
main()

## Display Results

In [ ]:
from IPython.display import Image, display
import glob
from pathlib import Path

# Display generated plots
output_dir = Path("experiments/out/bandgaps")

print("DoF and Beta Parameter Fits:")
display(Image(str(output_dir / "bandgaps_fits.png")))

print("\nModel Comparison with Baselines:")
display(Image(str(output_dir / "baseline_comparison.png")))

# Display numerical results
print("\nNumerical Results:")
with open(output_dir / "results.txt", 'r') as f:
    print(f.read())

## Optional: Experiment Parameters

You can modify these parameters to explore different aspects of the DoF law fitting:

In [ ]:
# Custom parameters for exploration
import numpy as np
import matplotlib.pyplot as plt
from tacc.micro.bandgaps import fit_dof_law, fit_beta

# Try different noise levels
noise_levels = [0.01, 0.05, 0.1, 0.2]
fitted_params = []

for noise in noise_levels:
    np.random.seed(42)
    N_values = np.linspace(0.5, 1.5, 50)
    true_a = 2.0
    DoF_values = np.exp(-true_a * (1 - N_values))
    DoF_values += np.random.normal(0, noise, len(DoF_values))
    
    # Fit with custom data
    from scipy.optimize import curve_fit
    def dof_model(N, a):
        return np.exp(-a * (1 - N))
    
    popt, _ = curve_fit(dof_model, N_values, DoF_values)
    fitted_params.append(popt[0])
    print(f"Noise level {noise:.2f}: fitted a = {popt[0]:.3f} (error: {abs(popt[0] - true_a):.3f})")

# Plot noise sensitivity
plt.figure(figsize=(8, 6))
plt.plot(noise_levels, fitted_params, 'bo-', linewidth=2, markersize=8)
plt.axhline(y=2.0, color='r', linestyle='--', label='True value (a=2.0)')
plt.xlabel('Noise Level')
plt.ylabel('Fitted Parameter a')
plt.title('Noise Sensitivity Analysis')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Troubleshooting

**Common Issues:**
- If plots don't display, ensure matplotlib is properly installed
- If imports fail, check that the repository was cloned correctly
- For fitting errors, verify that the synthetic data generation is working

**Expected Output:**
- Two plots showing DoF law fits and baseline model comparisons
- Numerical results showing fitted parameters and model comparison metrics (AIC/BIC)
- Our exponential model should outperform polynomial baselines for this synthetic data